In [1]:
import json
import os
from tqdm import tqdm
from prompts import *
from utils.chat_api import *

In [2]:
def verify_qa_list(qa_list, dataset_with_agent_answer_verified):
    bs = 100
    try:
        with open(dataset_with_agent_answer_verified, "r") as f:
            sample_count = len(f.readlines())
    except Exception as e:
        print(f"Error reading dataset_with_agent_answer_verified: {dataset_with_agent_answer_verified}")
        print(e)
        sample_count = 0
    for i in tqdm(range(sample_count, len(qa_list), bs)):
        try:
            qa_list_batch = qa_list[i:i+bs]
            inputs = [
                [
                    {
                        "type": "text",
                        "content": json.dumps({
                            "question": qa["question"],
                            "ground_truth_answer": qa["answer"],
                            "agent_answer": qa["agent_answer"],
                        }),
                    },
                    {
                        "type": "text",
                        "content": prompt_agent_verify_answer,
                    },
                    {
                        "type": "text",
                        "content": "Now answer if the answer from the baseline is correct or not:",
                    },            
                ] for qa in qa_list_batch
            ]
            messages = [generate_messages(input) for input in inputs]
            model = "gpt-4o-2024-11-20"
            responses = parallel_get_response(model, messages)

            verify_results = responses[0]
            for qa, verify_result in zip(qa_list_batch, verify_results):
                qa["verify_result"] = verify_result
            
            with open(dataset_with_agent_answer_verified, "a") as f:
                for qa in qa_list_batch:
                    f.write(json.dumps(qa) + "\n")
        except Exception as e:
            raise RuntimeError(f"Error processing qa_list_batch: {i}") from e
                
def verify_qa_list_with_reasoning(qa_list, dataset_with_agent_answer_verified):
    bs = 100
    try:
        with open(dataset_with_agent_answer_verified, "r") as f:
            sample_count = len(f.readlines())
    except Exception as e:
        print(f"Error reading dataset_with_agent_answer_verified: {dataset_with_agent_answer_verified}")
        print(e)
        sample_count = 0
    for i in tqdm(range(sample_count, len(qa_list), bs)):
        try:
            qa_list_batch = qa_list[i:i+bs]
            inputs = [
                [
                    {
                        "type": "text",
                        "content": json.dumps({
                            "question": qa["question"],
                            "ground_truth_answer": qa["answer"],
                            "agent_answer": qa["agent_answer"],
                            "reasoning": qa["reasoning"],
                        }),
                    },
                    {
                        "type": "text",
                        "content": prompt_agent_verify_answer_with_reasoning,
                    },
                    {
                        "type": "text",
                        "content": "Now answer if the answer from the baseline is correct or not:",
                    },            
                ] for qa in qa_list_batch
            ]
            messages = [generate_messages(input) for input in inputs]
            model = "gpt-4o-2024-11-20"
            responses = parallel_get_response(model, messages)

            verify_results = responses[0]
            for qa, verify_result in zip(qa_list_batch, verify_results):
                qa["verify_result"] = verify_result
            
            with open(dataset_with_agent_answer_verified, "a") as f:
                for qa in qa_list_batch:
                    f.write(json.dumps(qa) + "\n")
        except Exception as e:
            raise RuntimeError(f"Error processing qa_list_batch: {i}") from e

In [ ]:
dataset_with_agent_answer_list = ["data/annotations/results/0415/small_test_with_agent_answer_0.jsonl", "data/annotations/results/0415/small_test_with_agent_answer_1.jsonl", "data/annotations/results/0415/small_test_with_agent_answer_2.jsonl", "data/annotations/results/0415/small_test_with_agent_answer_3.jsonl", "data/annotations/results/0415/small_test_with_agent_answer_4.jsonl"]
dataset_with_agent_answer_verified_list = ["data/annotations/results/0416/small_test_with_agent_answer_verified_0.jsonl", "data/annotations/results/0416/small_test_with_agent_answer_verified_1.jsonl", "data/annotations/results/0416/small_test_with_agent_answer_verified_2.jsonl", "data/annotations/results/0416/small_test_with_agent_answer_verified_3.jsonl", "data/annotations/results/0416/small_test_with_agent_answer_verified_4.jsonl"]

for dataset_with_agent_answer, dataset_with_agent_answer_verified in zip(dataset_with_agent_answer_list, dataset_with_agent_answer_verified_list):
    qa_list = []
    
    with open(dataset_with_agent_answer, "r") as f:
        for line in f:
            qa = json.loads(line)
            qa_list.append(qa)

    qa_list = verify_qa_list(qa_list, dataset_with_agent_answer_verified)

In [ ]:
# calculate accuracy
dataset_with_agent_answer_verified = "data/annotations/results/0416/baseline_blindly_answers_verified.jsonl"
total = 0
correct = 0

with open(dataset_with_agent_answer_verified, "r") as f:
    for line in f:
        qa = json.loads(line)
        total += 1
        if qa["verify_result"].lower().startswith("yes"):
            correct += 1

print(f"Accuracy: {correct / total}")

In [3]:
# calculate pass@k
k = 5

filtered_questions = []
with open("data/annotations/results/0416/baseline_blindly_answers_verified.jsonl", "r") as f:
    for line in f:
        sample = json.loads(line)
        if sample["verify_result"].lower().startswith("yes"):
            filtered_questions.append(sample["question"])

print(len(filtered_questions))

with open("/mnt/bn/videonasi18n/ypan/Project/3_MM_Agents/dataset/verified_small_test.json", "r") as f:
    samples = json.load(f)
    for sample in samples:
        if sample["verify_result"].lower().startswith("yes") and sample["question"] not in filtered_questions:
            filtered_questions.append(sample["question"])

print(len(filtered_questions))


dataset_with_agent_answer_verified = (
    "data/annotations/results/forcing_answer/small_test_with_agent_answer_verified_{round}.jsonl"
)

results = [dataset_with_agent_answer_verified.format(round=i) for i in range(k)]
result_files = [open(result, "r") for result in results]

total = 0
correct = 0

# Read all QA pairs line by line across k files
while True:
    qa_attempts = []
    # Try to read next line from each file
    for f in result_files:
        line = f.readline()
        if line:
            qa = json.loads(line)
            question = qa["question"]
            qa_attempts.append(qa["verify_result"].lower().startswith("yes"))

    # If we've reached end of files, break
    if not qa_attempts:
        break

    # Count as correct if any attempt was successful
    if question not in filtered_questions:
        total += 1
        if any(qa_attempts):
            correct += 1

# Close all files
for f in result_files:
    f.close()

print(f"Pass@{k}: {correct / total}")

113
164
Pass@5: 0.7240566037735849


In [18]:
with open("/mnt/bn/videonasi18n/ypan/Project/3_MM_Agents/evaluation/vl_eval.json", "r") as f:
    videos = json.load(f)

total = 0
correct = 0

for video in videos["video_results"]:
    qas = video["qa_results"]
    for qa in qas:
        if qa["question"] not in filtered_questions:
            total += 1
            if qa["is_correct"]:
                correct += 1

print(correct/total)

0.38529411764705884
